In [1]:
import warnings 
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Activation,Dropout,Conv2D,Flatten,MaxPool2D,Reshape,GlobalAveragePooling2D,InputLayer
from tensorflow.keras.applications.resnet50 import preprocess_input 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau

In [3]:
IMAGE_SIZE=[229,229]
train_path='../input/natural-disaster-intensity/dataset/train_set'
test_path='../input/natural-disaster-intensity/dataset/test_set'

In [4]:
train_data_gen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.30)
test_data_gen=ImageDataGenerator(rescale=1./255,validation_split=0.30)

In [5]:
training_set=train_data_gen.flow_from_directory(train_path,target_size=(229,229),batch_size=100,class_mode='categorical',shuffle=True,color_mode='rgb',subset='training')
testing_set=test_data_gen.flow_from_directory(test_path,target_size=(229,229),batch_size=100,class_mode='categorical',shuffle=True,color_mode='rgb',subset='validation')

Found 521 images belonging to 4 classes.
Found 58 images belonging to 4 classes.


In [40]:
model=Sequential()

In [41]:
model.add(Conv2D(32,(3,3),input_shape=(229,229,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dense(units=4,activation='softmax'))

In [42]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 227, 227, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 113, 113, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 111, 111, 32)      9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 55, 55, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 96800)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               12390528  
_________________________________________________________________
dense_1 (Dense)              (None, 4)               

In [43]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [44]:
model.fit_generator(
    generator=training_set,steps_per_epoch=len(training_set),
    epochs=20,validation_data=testing_set,validation_steps=len(testing_set))

Epoch 1/20
6/6 [==============================] - 40s 6s/step - loss: 7.4400 - accuracy: 0.2745 - val_loss: 4.7555 - val_accuracy: 0.1379
Epoch 2/20
6/6 [==============================] - 30s 6s/step - loss: 1.6385 - accuracy: 0.3186 - val_loss: 1.4745 - val_accuracy: 0.3103
Epoch 3/20
6/6 [==============================] - 30s 5s/step - loss: 1.2937 - accuracy: 0.4127 - val_loss: 1.3408 - val_accuracy: 0.4138
Epoch 4/20
6/6 [==============================] - 31s 5s/step - loss: 1.2657 - accuracy: 0.4088 - val_loss: 1.2716 - val_accuracy: 0.6034
Epoch 5/20
6/6 [==============================] - 31s 5s/step - loss: 1.1531 - accuracy: 0.5125 - val_loss: 1.2474 - val_accuracy: 0.4138
Epoch 6/20
6/6 [==============================] - 31s 5s/step - loss: 1.0124 - accuracy: 0.5336 - val_loss: 1.1508 - val_accuracy: 0.5862
Epoch 7/20
6/6 [==============================] - 31s 5s/step - loss: 0.8680 - accuracy: 0.6468 - val_loss: 1.0622 - val_accuracy: 0.6034
Epoch 8/20
6/6 [==================

In [45]:
model.save("nature.h5")

In [48]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("nature.h5")

In [52]:
a = ['Cyclone','Earthquake','Flood','Wildfire']
img=image.load_img('../input/natural-disaster-intensity/dataset/test_set/Cyclone/876.jpg',
                  target_size=(229,229))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred=np.argmax(model.predict(x))
a[pred]

'Cyclone'

In [49]:
testing_set.class_indices

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}